# PPO2 on Solo8 v2 Vanilla w/ Fixed Timestamp & Vectorized Environments
Vectorized environment test

## Ensure that Tensorflow is using the GPU

In [1]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


## Define Experiment Tags

In [2]:
TAGS = ['solov2vanilla', 'gpu', 'home_pos_task']

## Get Solo Environment Configuration

Import the relevant libraries + rewards & observations

In [3]:
from gym_solo.envs import solo8v2vanilla
from gym_solo.core import obs
from gym_solo.core import rewards
from gym_solo.core import termination as terms

import gym
import gym_solo

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Create the config for the enviornment

In [4]:
env_config = solo8v2vanilla.Solo8VanillaConfig()

## Parse CLI arguments and register w/ wandb

This experiment will be using the auto trainer to handle all of the hyperparmeter running

In [5]:
from auto_trainer import params
import auto_trainer

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Create a basic config. Give the robot a total of 60 seconds simulation time to learn how to stand.

In [6]:
config = params.BaseParameters().parse()

config.episodes = 10
# config.episode_length = 10 / env_config.dt
config.episode_length = 1000

# auto_trainer.trainer._WANDB=False
config, run = auto_trainer.get_synced_config(config, TAGS)
config

wandb: Currently logged in as: agupta231 (use `wandb login --relogin` to force relogin)
/usr/local/lib/python3.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


{'episodes': 10, 'episode_length': 1000, 'policy': 'MlpPolicy', 'algorithm': 'PPO2'}

## Setup Environment
Add the following inputs to the robot / environment:

**Observations**
- TorsoIMU
- Motor encoder current values

**Reward**
- How upright the TorsoIMU is. Valued in $[-1, 1]$

**Termination Criteria**
- Terminate after $n$ timesteps

In [7]:
def make_env(length):
    def _init():
        env = gym.make('solo8vanilla-v0', config=env_config)

        env.obs_factory.register_observation(obs.TorsoIMU(env.robot))
        env.obs_factory.register_observation(obs.MotorEncoder(env.robot))

        # env.reward_factory.register_reward(1, rewards.UprightReward(env.robot))
        env.reward_factory.register_reward(1, rewards.HomePositionReward(env.robot))

        env.termination_factory.register_termination(terms.TimeBasedTermination(length))
        
        return env
    return _init

## Learning

Hand-vectorize the environment. Note that this should be abstracted out into its own utility method eventually.

In [8]:
from stable_baselines.common.vec_env import SubprocVecEnv

NUM_WORKERS = 4
env = SubprocVecEnv([make_env(config.episode_length) for _ in range(NUM_WORKERS)])
env

Train on the vectorized environment instead

In [9]:
model, config, run = auto_trainer.train(env, config, TAGS, log_freq=100,
                                        full_logging=False, run=run)





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




-------------------------------------
| approxkl           | 0.0020724821 |
| clipfrac           | 0.012207031  |
| explained_variance | 0.0126       |
| fps                | 569          |
| n_updates          | 1            |
| policy_entropy     | 17.029213    |
| policy_loss        | -0.008221899 |
| serial_timesteps   | 128          |
| time_elapsed       | 0.000262     |
| total_timesteps    | 512          |
| value_loss         | 16.60308     |
-------------------------------------



global_step,10208
_timestamp,1612639982.79303
loss/entropy_loss,17.08247
loss/policy_gradient_loss,-0.00204
loss/value_function_loss,81.94783
loss/approximate_kullback-leibler,0.00066
loss/clip_factor,0.0
loss/loss,40.80105
input_info/discounted_rewards,19.10188
input_info/learning_rate,0.00025
input_info/advantage,0.0


global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss/entropy_loss,▁▁▁▁▁▂▂▃▃▃▄▄▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▆▆▆▆▇▇████
loss/policy_gradient_loss,▇▃▂▇▁▆▅▆▆▆▆▆▆▇▆▆▇▆▄▆▅▇▇▆▆▆▆▄▆▆▇█▆▅▆▃▆▂▇▅
loss/value_function_loss,▁▁▁▄▃▆▆█▇██▇▇▆▆▃▃▁▁▃▃▆▆▇▇▆▇▆▅▆▄▄▁▁▁▁▄▄▆▆
loss/approximate_kullback-leibler,▁▃█▂▄▂▃▁▁▁▁▁▁▁▂▁▂▁▂▁▃▁▂▁▁▁▁▁▁▁▁▁▁▂▁▃▂▆▁▂
loss/clip_factor,▁▁█▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▁▁
loss/loss,▁▁▁▄▃▆▆█▇██▇▇▆▆▃▃▁▁▃▃▆▆▇▇▆▇▆▅▆▄▄▁▁▁▁▄▄▆▆
input_info/discounted_rewards,▁▁▁▄▃▅▅▆▆▆▆▆▆▅▅▄▄▃▃▅▅▇▆▇▇▆▆▆▅▆▅▅▃▃▅▅▇▇██
input_info/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
input_info/advantage,▇▃▆█▄▂▅▅▃▅▅▃▁▅▅▅▅▅▅▇▅▃▂▂▇▃▅▆▅▅▄▂▅▄▇▁▅█▁▅


In [10]:
model